In [2]:
from os import environ
from pathlib import Path
from json import loads, dumps
from random import choice
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10, temperature = 1):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query, temperature)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [3]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [4]:
def convertSentenceNetworkToVisNetworkJSON(sentenceNetwork):
    """
    Converts a sentence network to a vis network JSON object
    Format of sentenceNetwork:
    {
        <id string of the sentence>: [<sentence compount 1>, <sentence compount 2>, ...],
        ...
    }
    where sentence compount is eather a string,
        which means that it is not a concept but a predicate, adjective, etc.
    or a list of the form:
        [<id string of the concept>, <name string of the concept>] or
        [<id string of the concept>]
        which means that it is a concept
    """
    addedNodeIds = set()
    nodes = []
    edges = []
    internalIdCounter = 0
    def getNextInternalId():
        nonlocal internalIdCounter
        internalIdCounter += 1
        return str(internalIdCounter)
    # Create the sentence head nodes
    for sentenceID in sentenceNetwork.keys():
        sentenceID = "_" + sentenceID
        assert not sentenceID in addedNodeIds
        nodes.append({"id": sentenceID, "shape": "diamond", "size": 20, "color": {"background": "rgb(255, 70, 70)"}})
        addedNodeIds.add(sentenceID)
    # Create the sentence tails
    for sentenceID, sentenceCompounds in sentenceNetwork.items():
        lastCompointID = "_" + sentenceID
        for sentenceCompound in sentenceCompounds:
            if isinstance(sentenceCompound, str):
                id = getNextInternalId()
                nodes.append({"id": id, "label": sentenceCompound, "shape": "box", "color": {"background": "rgb(255, 230, 34)"}, "font": {"size": 15}})
            else:
                linkNodeId = getNextInternalId()
                nodes.append({"id": linkNodeId, "shape": "dot", "size": 10, "color": {"background": "rgb(255, 130, 70)"}})
                id, name, newDefined = (sentenceCompound + [None, None])[:3]
                id = "_" + id
                if not id in addedNodeIds:
                    nodes.append({"id": id, "label": name, "shape": "ellipse", "size": 40, "color": {"background": "rgb(90, 230, 255)", "border": "rgb(0, 0, 0)"}, "font": {"size": 20}})
                    addedNodeIds.add(id)
                else:
                    name = None
                if newDefined:
                    edges.append({"from": linkNodeId, "to": id, "arrows": "from", "color": {"color": "gray"}, "value" : 3})
                else:
                    edges.append({"from": linkNodeId, "to": id, "arrows": "from", "color": {"color": "gray"}})
                id = linkNodeId
            edges.append({"from": lastCompointID, "to": id, "value": 3, "arrows": "to", "color": {"color": "rgb(255, 180, 70)"}})
            lastCompointID = id
    return {"nodes": nodes, "edges": edges}

In [5]:
def saveSentenceNetworkToVisNetworkHTMLFile(sentenceNetwork, htmlFilePath):
    safeVisNetworkJSONToHTMLFile(convertSentenceNetworkToVisNetworkJSON(sentenceNetwork), htmlFilePath)

In [76]:
exampleSentenceNetwork = {
    "1": [["2", "The electron"], "is a", ["3"]],
    "3": [["4", "Subatomic particle"], "with a", ["5", "negative electric charge"]],
    "6": [["2"], "are thought to be", ["7", "elementary particles"]]
}
saveSentenceNetworkToVisNetworkHTMLFile(exampleSentenceNetwork, "output.html")

In [6]:
class SentecnceNetwork:
    def __init__(self):
        self.sentences = set()
        self.concepts = set()
        self.idCounter = 0
    def sentence(self, *sentenceCompounds):
        sentence = Sentence(self, sentenceCompounds, self.idCounter)
        self.idCounter += 1
        self.sentences.add(sentence)
        return sentence
    def concept(self, name, newDefined = False):
        concept = Concept(self, name, self.idCounter, newDefined)
        self.idCounter += 1
        self.concepts.add(concept)
        return concept
    def getNetwork(self):
        network = {}
        for sentence in self.sentences:
            network[str(sentence.sentenceID)] = [compound if isinstance(compound, str) else [str(compound.sentenceID)] if isinstance(compound, Sentence) else [str(compound.id), compound.name, compound.definingSentence == sentence] for compound in sentence.compounds]
        return network
        
class Sentence:
    def __init__(self, sentenceNetwork, sentenceCompounds, sentenceID):
        self.sentenceNetwork = sentenceNetwork
        self.sentenceID = sentenceID
        self.compounds = sentenceCompounds
        for compound in sentenceCompounds:
            if isinstance(compound, Concept):
                if compound.newDefined:
                    compound.definingSentence = self
                    compound.newDefined = False

class Concept:
    def __init__(self, sentenceNetwork, name, id, newDefined = False):
        self.sentenceNetwork = sentenceNetwork
        self.name = name
        self.id = id
        self.definingSentence = None
        self.newDefined = newDefined

In [7]:
sn = SentecnceNetwork()

subatomicParticle = sn.concept("Subatomic particle")
subatPartWithNeg = sn.sentence(subatomicParticle, "with a", sn.concept("Negative electric charge"))

electron = sn.concept("Electron")
sn.sentence(electron, "is a", subatPartWithNeg)
elementaryParticles = sn.concept("Elementary particles")
electronThoughtToBe = sn.sentence(electron, "is thought to be", elementaryParticles)

firstGenerationLepton = sn.concept("First generation")
firstGenLepOfPartFam = sn.sentence(firstGenerationLepton, "of", sn.concept("Lepton particle family"))
sn.sentence(electron, "belongs to", firstGenLepOfPartFam)

knownComponentsOrSubstructure = sn.sentence("Known", sn.concept("Components"), "or", sn.concept("Substructure"))
electronHasNo = sn.sentence(electron, "has no", knownComponentsOrSubstructure)
sn.sentence(electronThoughtToBe, "because", electronHasNo)

mass = sn.concept("Mass")
electronsMass = sn.sentence(mass, "of the", electron)
proton = sn.concept("Proton")
fractionOfProtonMass = sn.sentence(sn.concept("1/1836"), "of the", mass, "of the", proton)
sn.sentence(electronsMass, "is approximately", fractionOfProtonMass)

spinOfHalfInteger = sn.sentence(sn.concept("Spin"), "of", sn.concept("half-integer value"))
unitsOfTheReducedPlancksConstant = sn.sentence(sn.concept("Units"), "of the", sn.concept("Reduced Planck's constant"))
sn.sentence(spinOfHalfInteger, "is expressed in", unitsOfTheReducedPlancksConstant)

quantMechPropOfElectron = sn.sentence(sn.concept("Quantum mechanical properties"), "of the", electron)
sn.sentence(quantMechPropOfElectron, "include", spinOfHalfInteger)

sn.sentence("The", electron, "is a", sn.concept("Fermion"))
twoElectrons = sn.sentence("Two", electron)
canNotOccupy = sn.sentence(twoElectrons, "cannot occupy the same", sn.concept("Quantum state"))
sn.sentence(canNotOccupy, "because", sn.concept("Pauli exclusion principle"))

particle = sn.concept("Particle")
wave = sn.concept("Wave")
particlesAndWaves = sn.sentence(particle, "and", wave)
electronAsWellAsElementaryParticles = sn.sentence(electron, "as well as other", elementaryParticles)
property = sn.concept("Property")
sn.sentence(electronAsWellAsElementaryParticles, "exhibits" , property, "of", particlesAndWaves)

sn.sentence(electron, "can collide with", particle)
sn.sentence(electron, "can be defracted like", sn.concept("Light"))

neutronAndProton = sn.sentence(sn.concept("Neutron"), "and", proton)
wavePropertiesElectron = sn.sentence(sn.concept("Wave"), property, "of", electron)
wavePropertiesProtonAndNeutron = sn.sentence(sn.concept("Wave"), property, "of", neutronAndProton)
easierToObserve = sn.sentence(wavePropertiesElectron, "are easier to observe than", wavePropertiesProtonAndNeutron)

haveLowerMass = sn.sentence(electron, "have a lower", mass, "than", neutronAndProton)
longerDeBroglieWavelength = sn.sentence(electron, "have a longer", sn.concept("De Broglie wavelength"), "than", neutronAndProton, "for given", sn.concept("Energy"))
massAndThereforeWavelength = sn.sentence(haveLowerMass, "and therefore", longerDeBroglieWavelength)
sn.sentence(easierToObserve, "because", massAndThereforeWavelength)

saveSentenceNetworkToVisNetworkHTMLFile(sn.getNetwork(), "output.html")

In [ ]:
electronText = """
The electron is a subatomic particle with a negative one elementary electric charge Electrons belong to the first generation of the lepton particle family,[14] and are generally thought to be elementary particles because they have no known components or substructure.[1] The electron's mass is approximately 1/1836 that of the proton.[15] Quantum mechanical properties of the electron include an intrinsic angular momentum (spin) of a half-integer value, expressed in units of the reduced Planck constant, ħ. Being fermions, no two electrons can occupy the same quantum state, per the Pauli exclusion principle.[14] Like all elementary particles, electrons exhibit properties of both particles and waves: They can collide with other particles and can be diffracted like light. The wave properties of electrons are easier to observe with experiments than those of other particles like neutrons and protons because electrons have a lower mass and hence a longer de Broglie wavelength for a given energy.

Electrons play an essential role in numerous physical phenomena, such as electricity, magnetism, chemistry, and thermal conductivity; they also participate in gravitational, electromagnetic, and weak interactions.[16] Since an electron has charge, it has a surrounding electric field; if that electron is moving relative to an observer, the observer will observe it to generate a magnetic field. Electromagnetic fields produced from other sources will affect the motion of an electron according to the Lorentz force law. Electrons radiate or absorb energy in the form of photons when they are accelerated.

Laboratory instruments are capable of trapping individual electrons as well as electron plasma by the use of electromagnetic fields. Special telescopes can detect electron plasma in outer space. Electrons are involved in many applications, such as tribology or frictional charging, electrolysis, electrochemistry, battery technologies, electronics, welding, cathode-ray tubes, photoelectricity, photovoltaic solar panels, electron microscopes, radiation therapy, lasers, gaseous ionization detectors, and particle accelerators. """